---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
#%matplotlib inline
#import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import math

from sklearn.ensemble import RandomForestClassifier
from adspy_shared_utilities import plot_class_regions_for_classifier_subplot
from sklearn import preprocessing


In [2]:
train_df = pd.read_csv('train.csv',encoding='ISO-8859-1')

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
test_df = pd.read_csv('test.csv',encoding='ISO-8859-1')

In [4]:
train_df.shape, test_df.shape

((250306, 34), (61001, 27))

In [5]:
train_df.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [6]:
train_df.columns, test_df.columns

(Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
        'violation_street_number', 'violation_street_name',
        'violation_zip_code', 'mailing_address_str_number',
        'mailing_address_str_name', 'city', 'state', 'zip_code',
        'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
        'violation_code', 'violation_description', 'disposition', 'fine_amount',
        'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
        'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
        'payment_date', 'payment_status', 'collection_status',
        'grafitti_status', 'compliance_detail', 'compliance'],
       dtype='object'),
 Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
        'violation_street_number', 'violation_street_name',
        'violation_zip_code', 'mailing_address_str_number',
        'mailing_address_str_name', 'city', 'state', 'zip_code',
        'non_us_str_code', 'country', '

In [7]:
train_df = train_df[train_df.compliance.notnull()] #filter out compliance == NaN

In [8]:
train_df.head() 

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
5,22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478.0,NORTHFIELD,NaN,2755.0,E. 17TH,LOG BEACH,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
6,18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Deerwood Development Group Inc, Deer",8027.0,BRENTWOOD,NaN,476.0,Garfield,Clinton,...,0.0,855.0,0.0,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
7,18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,...,0.0,140.0,0.0,140.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0


In [9]:
len(train_df[train_df.compliance==1]),len(train_df[train_df.compliance==0])

(11597, 148283)

In [10]:
#plt.scatter(train_df['late_fee'], train_df['compliance'])

In [11]:
train_df = train_df.set_index('ticket_id')  # set index as ticket_id
test_df = test_df.set_index('ticket_id')

In [12]:
len(train_df)

159880

In [13]:
#preProcLabel = pd.DataFrame(index = train_df.index)

labels = ['agency_name', 'violation_street_name', 'disposition']

le = preprocessing.LabelEncoder()
for label in labels:     
    train_df[label] = le.fit_transform(train_df[label])
    test_df[label] = le.fit_transform(test_df[label])

In [14]:
train_df

,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
ticket_id,,,,,,,,,,,,,,,,,,,,,
22056,0,"Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,1571,NaN,3.0,S. WICKER,CHICAGO,IL,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
27586,0,"Williams, Darrin","Michigan, Covenant House",4311.0,290,NaN,2959.0,Martin Luther King,Detroit,MI,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
22046,0,"Sims, Martinzie","KASIMU, UKWELI",6478.0,1186,NaN,2755.0,E. 17TH,LOG BEACH,CA,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
18738,0,"Williams, Darrin","Deerwood Development Group Inc, Deer",8027.0,202,NaN,476.0,Garfield,Clinton,MI,...,0.0,855.0,0.0,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
18735,0,"Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,1151,NaN,8228.0,Mt. Elliott,Detroit,MI,...,0.0,140.0,0.0,140.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
18733,0,"Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,1151,NaN,8228.0,Mt. Elliott,Detroit,MI,...,0.0,140.0,0.0,140.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
28204,0,"Williams, Darrin","Inc, Nanno",15307.0,1425,NaN,1537.0,E. Seven Mile,Detroit,MI,...,0.0,855.0,0.0,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
18743,0,"Williams, Darrin","Gardner Resale, GAR",9100.0,1579,NaN,91.0,Van Dyke,Detroit,MI,...,0.0,855.0,0.0,855.0,NaN,NO PAYMENT APPLIED,IN COLLECTION,NaN,non-compliant by no payment,0.0
18741,0,"Williams, Darrin","Hardaway, Kevin",20024.0,1406,NaN,224.0,Schaefer,Detroit,MI,...,0.0,855.0,0.0,855.0,NaN,NO PAYMENT APPLIED,IN COLLECTION,NaN,non-compliant by no payment,0.0


In [15]:
X_train = train_df[['fine_amount', 'judgment_amount', 'admin_fee', 
            'state_fee', 'late_fee', 'agency_name', 'violation_street_name', 'disposition']]

In [16]:
X_train.shape

(159880, 8)

In [17]:
X_train.head()

,fine_amount,judgment_amount,admin_fee,state_fee,late_fee,agency_name,violation_street_name,disposition
ticket_id,,,,,,,,
22056,250.0,305.0,20.0,10.0,25.0,0,1571,2
27586,750.0,855.0,20.0,10.0,75.0,0,290,3
22046,250.0,305.0,20.0,10.0,25.0,0,1186,2
18738,750.0,855.0,20.0,10.0,75.0,0,202,2
18735,100.0,140.0,20.0,10.0,10.0,0,1151,2


In [18]:
y_train = train_df.iloc[:, -1]

In [19]:
clf = RandomForestClassifier(n_estimators =50, max_features=2, min_samples_leaf=10, 
    class_weight="balanced", n_jobs=-1, oob_score=True, verbose=True).fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    4.2s finished


In [20]:
y_hat = clf.predict(X_train)

clf.score(X_train, y_train)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.6s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.6s finished


0.85865023767825865

In [21]:
clf.feature_importances_

array([ 0.04125229,  0.11832008,  0.        ,  0.        ,  0.3307672 ,
        0.01863972,  0.27243431,  0.2185864 ])

In [22]:
y_train_proba = clf.predict_proba(X_train)[:, 1]

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.6s finished


In [23]:
y_train_proba[:10]

array([ 0.32598853,  0.53910316,  0.3047148 ,  0.30973684,  0.62918293,
        0.62918293,  0.27649294,  0.04300937,  0.09220863,  0.04300937])

In [24]:
X_test = test_df[['fine_amount', 'judgment_amount', 'admin_fee', 
                  'state_fee', 'late_fee', 'agency_name', 'violation_street_name', 'disposition']]

In [25]:
X_test.head()

,fine_amount,judgment_amount,admin_fee,state_fee,late_fee,agency_name,violation_street_name,disposition
ticket_id,,,,,,,,
284932,200.0,250.0,20.0,10.0,20.0,1,1181,5
285362,1000.0,1130.0,20.0,10.0,100.0,1,471,5
285361,100.0,140.0,20.0,10.0,10.0,1,471,5
285338,200.0,250.0,20.0,10.0,20.0,1,255,5
285346,100.0,140.0,20.0,10.0,10.0,1,255,5


In [26]:
y_proba = clf.predict_proba(X_test)[:, 1]
y_test_hat = clf.predict(X_test)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.2s finished


In [27]:
compliance = pd.Series(data = y_proba, index = X_test.index)

In [28]:
def blight_model():
    
    return compliance

In [29]:
len(compliance)

61001